In [1]:
#  Importing libraries and module and some setting for notebookimport pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct  # Leading Juice for us
import time
from fuzzywuzzy import fuzz
import fuzzywuzzy
import pandas as pd
import tqdm
import os

In [2]:
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    string = fuzz.utils.full_process(string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)
    return csr_matrix((data,indices,indptr),shape=(M,N))


# unpacks the resulting sparse matrix
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

## PreQin

In [3]:
# Henter downloaded preqin data
preqin = pd.read_excel(r'C:\Users\jacob\Documents\Polit\Seminar\Data\PreQin_Masterark.xlsx', sheet_name='Preqin_Export', header = 0)
#preqin = preqin[ preqin['DEAL DATE'] > '2016-01-01']

preqin['Company'] = [fuzz.utils.full_process(com) for com in preqin['PORTFOLIO COMPANY']]
preqin['Company'] = [re.sub(r'[,-./]|\sBD',r'', com) for com in preqin['Company']]

preqin_c_unique = list(preqin['Company'].unique())
preqin_countries = preqin['PORTFOLIO COMPANY COUNTRY'].unique()

In [5]:
# Loops matching over each patent category

for file in tqdm.tqdm(os.scandir(r'C:\Users\jacob\Documents\Polit\Seminar\Data\Orbis\Wibo')):
    c = file.name
    category = c.replace('.xlsx', '')
    orbis= pd.read_excel(file, sheet_name='Results', header = 0)
    orbis['Company'] = [fuzz.utils.full_process(com) for com in orbis['Company name Latin alphabet']]
    orbis['Company'] = [re.sub(r'[,-./]|\sBD',r'', com) for com in orbis['Company']]

    orbis_c_unique = list(orbis['Company'].unique())

    ######################################################################################################################
    # Cleaning the Orbis countries to match Preqin

    orbis['Country_'] = orbis['Country'].fillna('')
    for i in range(0,len(orbis)):
        c = orbis['Country_'].iloc[i]

        if 'United States of America' in c:
            orbis['Country_'].iloc[i] = 'US'

        if 'France' in c:
            orbis['Country_'].iloc[i] = 'France'

        if 'Denmark' in c:
            orbis['Country_'].iloc[i] = 'Denmark'

        if 'United Kingdom' in c:
            orbis['Country_'].iloc[i] = 'UK'

        if 'Russian Federation' in c:
            orbis['Country_'].iloc[i] = 'Russia'

        if 'Republic of Moldova' in c:
            orbis['Country_'].iloc[i] = 'Moldova'

    #######################################################################################################################
    # Match names in Preqin and Orbis for each country

    orbis_countries = list(orbis['Country_'].unique())

    same = []
    not_ = []
    for i in preqin_countries:
        if i in orbis_countries:
            same.append(i)
        else:
            not_.append(i)

    in_ = []

    for c_ in same:
        orbis_temp = orbis[ orbis['Country_'] == c_]
        preqin_temp = preqin[ preqin['PORTFOLIO COMPANY COUNTRY'] == c_]

        preqin_company = preqin_temp['Company'].unique()
        orbis_company = orbis_temp.iloc[:, 1].unique()

        del preqin_temp, orbis_temp

        collected = list(preqin_company) + list(orbis_company)

        del orbis_company

        collected = [fuzz.utils.full_process(com) for com in collected]

        vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
        tf_idf_matrix_c = vectorizer.fit_transform(collected)

        #  Top 4 with similarity above 0.8
        matches = awesome_cossim_top(tf_idf_matrix_c, tf_idf_matrix_c.transpose(), 3, 0.8) 

        matches_df = get_matches_df(matches, collected, top = False)
        matches_df = matches_df[matches_df['similairity'] < 0.99999] # For removing all exact matches

        del matches, collected

        for m in range(0, len(matches_df)):
            #if (matches_df.iloc[m, 0] == matches_df.iloc[m, 1]) & (matches_df.iloc[m, 0] in preqin_company):
            #    in_.append(matches_df.iloc[m, 0])
            if (matches_df.iloc[m, 0] in preqin_company) & (matches_df.iloc[m, 1] not in preqin_company):
                in_.append(matches_df.iloc[m, 0])
            elif (matches_df.iloc[m, 0] not in preqin_company) & (matches_df.iloc[m, 1] in preqin_company):
                in_.append(matches_df.iloc[m, 1])
        
        del matches_df

    ######################################################################################################################
    # 'Other' countries

    orbis_temp2 = orbis[ ~orbis['Country_'].isin(same)] 
    preqin_temp2 = preqin[ ~preqin['PORTFOLIO COMPANY COUNTRY'].isin(same)]

    preqin_company = preqin_temp2['Company'].unique()
    orbis_company = orbis_temp2.iloc[:, 1].unique()

    del preqin_temp2, orbis_temp2

    collected = list(preqin_company) + list(orbis_company)

    del orbis_company

    collected = [fuzz.utils.full_process(com) for com in collected]

    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
    tf_idf_matrix_c = vectorizer.fit_transform(collected)

    #  Top 4 with similarity above 0.8
    matches = awesome_cossim_top(tf_idf_matrix_c, tf_idf_matrix_c.transpose(), 3, 0.8) 

    matches_df = get_matches_df(matches, collected, top = False)
    matches_df = matches_df[matches_df['similairity'] < 0.99999] # For removing all exact matches

    del matches, collected

    for m in range(0, len(matches_df)):
        #if (matches_df.iloc[m, 0] == matches_df.iloc[m, 1]) & (matches_df.iloc[m, 0] in preqin_company):
        #    in_.append(matches_df.iloc[m, 0])
        if (matches_df.iloc[m, 0] in preqin_company) & (matches_df.iloc[m, 1] not in preqin_company):
            in_.append(matches_df.iloc[m, 0])
        elif (matches_df.iloc[m, 0] not in preqin_company) & (matches_df.iloc[m, 1] in preqin_company):
            in_.append(matches_df.iloc[m, 1])

    del matches_df

    #######################################################################################################################
    # Identical names in PreQin and Orbis
    
    [in_.append(com) for com in preqin_c_unique if com in orbis_c_unique]

    #######################################################################################################################

    in_unique = list(dict.fromkeys(in_))
    print(category, len(in_), len(in_unique))

    preqin[category] =  np.where(preqin['Company'].isin(in_unique), 1, 0)

preqin_raw = preqin.copy()


0it [00:00, ?it/s]c:\Users\jacob\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
1it [01:53, 113.67s/it]

Analysis of biological materials 2612 1438


2it [07:04, 229.69s/it]

Audio-visual technology 3757 2009


3it [08:39, 168.15s/it]

...

Textile and paper machines 936 512


34it [2:36:30, 266.68s/it]

Thermal processes and apparatus 1039 575


35it [2:44:58, 282.81s/it]

Transport 2273 1234


In [2]:
preqin_raw = pd.read_excel(r'C:\Users\jacob\Documents\Polit\Seminar\Data\PreQin v2.3 c80.xlsx', header = 0)

In [3]:
preqin = preqin_raw.copy()

In [4]:
preqin['Electrical engineering']        =   preqin['Electrical machinery, apparatus, energy'] + \
                                            preqin['Audio-visual technology'] + \
                                            preqin['Telecommunications'] + \
                                            preqin['Digital communication'] + \
                                            preqin['Basic communication processes'] + \
                                            preqin['Computer technology'] + \
                                            preqin['IT methods for management'] + \
                                            preqin['Semiconductors']
preqin['Electrical engineering']        =   np.ceil(preqin['Electrical engineering'] / (preqin['Electrical engineering'] + 1))


preqin['Instruments']                   =   preqin['Optics'] + \
                                            preqin['Measurement'] + \
                                            preqin['Analysis of biological materials'] + \
                                            preqin['Control'] + \
                                            preqin['Medical technology']
preqin['Instruments']                   =   np.ceil(preqin['Instruments'] / (preqin['Instruments'] + 1))


preqin['Chemistry']                     =   preqin['Organic fine chemistry'] + \
                                            preqin['Biotechnology'] + \
                                            preqin['Pharmaceuticals'] + \
                                            preqin['Macromolecular chemistry, polymers'] + \
                                            preqin['Food chemistry'] + \
                                            preqin['Basic materials chemistry'] + \
                                            preqin['Materials, metallurgy'] + \
                                            preqin['Surface technology, coating'] + \
                                            preqin['Micro-structural and nano-technology-2'] + \
                                            preqin['Chemical engineering'] + \
                                            preqin['Environmental technology']
preqin['Chemistry']                     =   np.ceil(preqin['Chemistry'] / (preqin['Chemistry'] + 1))


preqin['Mechanical engineering']        =   preqin['Handling'] + \
                                            preqin['Machine Tools'] + \
                                            preqin['Engines, pumps, turbines-2'] + \
                                            preqin['Textile and paper machines'] + \
                                            preqin['Other special machines'] + \
                                            preqin['Thermal processes and apparatus'] + \
                                            preqin['Mechanical elements'] + \
                                            preqin['Transport']
preqin['Mechanical engineering']        =   np.ceil(preqin['Mechanical engineering'] / (preqin['Mechanical engineering'] + 1))


preqin['Other fields']                  =   preqin['Furniture, games'] + \
                                            preqin['Other consumer goods'] + \
                                            preqin['Civil engineering']
preqin['Other fields']                  =   np.ceil(preqin['Other fields'] / (preqin['Other fields'] + 1))


preqin['Deeptech']          =   preqin['Electrical engineering'] + \
                                preqin['Instruments'] + \
                                preqin['Chemistry'] + \
                                preqin['Mechanical engineering'] + \
                                preqin['Other fields'] 
preqin['Deeptech']          =   np.ceil(preqin['Deeptech'] / (preqin['Deeptech'] + 1))
preqin.to_excel('PreQin v2.3.1 c80.xlsx')


## Venturelisten

In [5]:
vl = pd.read_excel(r'C:\Users\jacob\Documents\Polit\Seminar\Data\Venturelisten v1.1.xlsx', header = 0)
kurser = pd.read_excel(r'C:\Users\jacob\Documents\Polit\Seminar\Data\Valutakurser.xlsx')

vl['Deeptech v1.1'] = vl['Deeptech']

vl['Company'] = [fuzz.utils.full_process(com) for com in vl['Name of company']]
vl['Company'] = [re.sub(r'[,-./]|\sBD',r'', com) for com in vl['Company']]
vl_c_unique = list(vl['Company'].unique())

In [6]:
for file in tqdm.tqdm(os.scandir(r'C:\Users\jacob\Documents\Polit\Seminar\Data\Orbis\Wibo')):
    c = file.name
    category = c.replace('.xlsx', '')
    orbis= pd.read_excel(file, sheet_name='Results', header = 0)
    
    orbis['Country_'] = orbis['Country'].fillna('')
    for i in range(0,len(orbis)):
        c = orbis['Country_'].iloc[i]

        if 'Denmark' in c:
            orbis['Country_'].iloc[i] = 'Denmark'

    orbis = orbis[ orbis['Country_'] == 'Denmark']

    orbis['Company'] = [fuzz.utils.full_process(com) for com in orbis['Company name Latin alphabet']]
    orbis['Company'] = [re.sub(r'[,-./]|\sBD',r'', com) for com in orbis['Company']]

    orbis_c_unique = list(orbis['Company'].unique())

    #######################################################################################################################
    # Match names in Preqin and Orbis

    vl_company = vl_c_unique
    orbis_company = orbis_c_unique

    collected = list(vl_company) + list(orbis_company)

    del orbis_company

    #collected = [fuzz.utils.full_process(com) for com in collected]

    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
    tf_idf_matrix_c = vectorizer.fit_transform(collected)

    #  Top 4 with similarity above 0.8
    matches = awesome_cossim_top(tf_idf_matrix_c, tf_idf_matrix_c.transpose(), 3, 0.80) 

    matches_df = get_matches_df(matches, collected, top = False)
    matches_df = matches_df[matches_df['similairity'] < 0.99999] # For removing all exact matches

    del matches, collected

    in_vl = []

    for m in range(0, len(matches_df)):
        #if (matches_df.iloc[m, 0] == matches_df.iloc[m, 1]) & (matches_df.iloc[m, 0] in vl_company):
        #    in_vl.append(matches_df.iloc[m, 0])
        if (matches_df.iloc[m, 0] in vl_company) & (matches_df.iloc[m, 1] not in vl_company):
            in_vl.append(matches_df.iloc[m, 0])
        elif (matches_df.iloc[m, 0] not in vl_company) & (matches_df.iloc[m, 1] in vl_company):
            in_vl.append(matches_df.iloc[m, 1])
        
    del matches_df

    #######################################################################################################################
    # Identical names in PreQin and Orbis
    
    [in_vl.append(com) for com in vl_c_unique if com in orbis_c_unique]

    #######################################################################################################################

    in_vl_unique = list(dict.fromkeys(in_vl))
    print(category, len(in_vl), len(in_vl_unique))

    vl[category] =  np.where(vl['Company'].isin(in_vl_unique), 1, 0)

vl_raw = vl.copy()

1it [00:04,  4.81s/it]

Analysis of biological materials 10 5


2it [00:15,  8.14s/it]

Audio-visual technology 8 3


3it [00:18,  5.93s/it]

...

Textile and paper machines 2 1


34it [07:52, 14.10s/it]

Thermal processes and apparatus 2 1


35it [08:18, 14.25s/it]

Transport 6 3


In [13]:
vl_raw = pd.read_excel(r'C:\Users\jacob\Documents\Polit\Seminar\Data\Venturelisten v2.3 c80.xlsx', header = 0)

In [24]:
vl = vl_raw.copy()

In [25]:
vl['Electrical engineering']        =   vl['Electrical machinery, apparatus, energy'] + \
                                            vl['Audio-visual technology'] + \
                                            vl['Telecommunications'] + \
                                            vl['Digital communication'] + \
                                            vl['Basic communication processes'] + \
                                            vl['Computer technology'] + \
                                            vl['IT methods for management'] + \
                                            vl['Semiconductors']
vl['Electrical engineering']        =   np.ceil(vl['Electrical engineering'] / (vl['Electrical engineering'] + 1))


vl['Instruments']                   =   vl['Optics'] + \
                                            vl['Measurement'] + \
                                            vl['Analysis of biological materials'] + \
                                            vl['Control'] + \
                                            vl['Medical technology']
vl['Instruments']                   =   np.ceil(vl['Instruments'] / (vl['Instruments'] + 1))


vl['Chemistry']                     =   vl['Organic fine chemistry'] + \
                                            vl['Biotechnology'] + \
                                            vl['Pharmaceuticals'] + \
                                            vl['Macromolecular chemistry, polymers'] + \
                                            vl['Food chemistry'] + \
                                            vl['Basic materials chemistry'] + \
                                            vl['Materials, metallurgy'] + \
                                            vl['Surface technology, coating'] + \
                                            vl['Micro-structural and nano-technology-2'] + \
                                            vl['Chemical engineering'] + \
                                            vl['Environmental technology']
vl['Chemistry']                     =   np.ceil(vl['Chemistry'] / (vl['Chemistry'] + 1))


vl['Mechanical engineering']        =   vl['Handling'] + \
                                            vl['Machine Tools'] + \
                                            vl['Engines, pumps, turbines-2'] + \
                                            vl['Textile and paper machines'] + \
                                            vl['Other special machines'] + \
                                            vl['Thermal processes and apparatus'] + \
                                            vl['Mechanical elements'] + \
                                            vl['Transport']
vl['Mechanical engineering']        =   np.ceil(vl['Mechanical engineering'] / (vl['Mechanical engineering'] + 1))


vl['Other fields']                  =   vl['Furniture, games'] + \
                                            vl['Other consumer goods'] + \
                                            vl['Civil engineering']
vl['Other fields']                  =   np.ceil(vl['Other fields'] / (vl['Other fields'] + 1))


vl['Deeptech']          =   vl['Electrical engineering'] + \
                                vl['Instruments'] + \
                                vl['Chemistry'] + \
                                vl['Mechanical engineering'] + \
                                vl['Other fields'] 
vl['Deeptech']          =   np.ceil(vl['Deeptech'] / (vl['Deeptech'] + 1))
vl.to_excel('Venturelisten v2.3 c80.xlsx')